
<style type="text/css">
pre code {
    display: block;
    unicode-bidi: embed;
    font-family: monospace;
    white-space: pre;
    max-height: 400px;
    overflow-x: scroll;
    overflow-y: scroll;
    }
</style>

In [ ]:
## This part is optional; it sets some printing options
## that make output look nicer.
from pprint import pprint as print
import pandas as pd
pd.set_option('display.width', 133)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', 5)

## Exercise: Retrieve exhibits data

## Exercise: Retrieve exhibits data {.smaller}
In this exercise you will retrieve information about the art
exhibitions at Harvard Art Museums from
`https://www.harvardartmuseums.org/visit/exhibitions`

1. Using a web browser (Firefox or Chrome recommended) inspect the
   page at `https://www.harvardartmuseums.org/visit/exhibitions`. Examine
   the network traffic as you interact with the page. Try to find
   where the data displayed on that page comes from.

In [ ]:
## TODO

2. Make a `get` request in Python to retrieve the data from the URL
   identified in step1.

In [ ]:
## TODO

3. Write a *loop* or *list comprehension* in Python to retrieve data
   for the first 5 pages of exhibitions data.

In [ ]:
## TODO

4. Bonus (optional): Arrange the data you retrieved into dict of
   lists. Convert it to a pandas `DataFrame` and save it to a `.csv`
   file.

In [ ]:
## TODO

## Exercise: parsing HTML
In this exercise you will retrieve information about the physical
layout of the Harvard Art Museums. The web page at
<https://www.harvardartmuseums.org/visit/floor-plan> contains this
information in HTML from.

1. Using a web browser (Firefox or Chrome recommended) inspect the
   page at `https://www.harvardartmuseums.org/visit/floor-plan`. Copy
   the `XPath` to the element containing list of facilities located on
   **level 1**. information. (HINT: the element if interest is a `ul`,
   i.e., an "unordered list" of class `ifp-floors__rooms`.)
   
   ![](./img/ex2_ul.png)
   
2. Make a `get` request in Python to retrieve the web page at
   <https://www.harvardartmuseums.org/visit/floor-plan>. Extract the
   content from your request object and parse it using `html.fromstring`
   from the `lxml` library.

In [ ]:
from lxml import html

floor_plan = requests.get('https://www.harvardartmuseums.org/visit/floor-plan')

floor_plan_html = html.fromstring(floor_plan.text)

3. Use the `XPath` you identified in step one to select the HTML list item
   containing level one information.

In [ ]:
level_one = floor_plan_html.xpath('/html/body/main/section/ul/li[5]/div[2]/ul')[0]

4. Use a *for loop* or *list comprehension* to iterate over the
   sub-elements of the list item you selected in the previous step and
   extract the text from each one.

In [ ]:
print([element.text_content() for element in level_one])

5. Bonus (optional): Extract the list of facilities available on each level.

In [ ]:
level_html = floor_plan_html.xpath('/html/body/main/section/ul/li')

level_info = [[element.text_content()
               for element in level.xpath('div[2]/ul')[0]]
              for level in level_html]

print(level_info)